# Planar Segmentation using NDT-RANSAC

This method is a derivative of https://github.com/xufana/RANSAC
It has been modified to work with my data, and translated from Russian to English

After the completion of LiDAR-DenseSeg this will be created as a fork of that repo, and the fork added as a submodule to this repo

In [ ]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import math
import scipy
from tqdm import tqdm_notebook

In [ ]:
data_path = '/data'
# For each point cloud (.xyz)

In [ ]:
def read_xyz(filename):
    xyz = np.loadtxt(filename, delimiter=' ', dtype=np.float32)
    return xyz

In [ ]:
s = 0.5
te = 0.01
A = point_cloud_in_numpy // s # dividing PCD into cubes
Q = np.array([0, 0, 0]) # flat NDT cells
Q_points = []
P = [] # other cells

In [ ]:
unique,counts=np.unique(A,axis=0, return_counts=True)
plt.figure(figsize=(15,5))
plt.bar(range(0, len(counts)), counts, width=1.5, color='g')
plt.show()

In [ ]:
def IRLS_find_norm(points):
    cov = np.cov(np.array(points).T)
    #print("cov = ", cov)
    lambd, e = np.linalg.eig(cov)
    e = e.T
    o = lambd.argsort()
    lambd = lambd[o]
    e = e[o]
    gamma = 1e-6
    k_w = 2.985
    n = e[0]
    g = np.mean(points, axis = 0)
    X_prev = np.zeros(3)
    for i in range(100):
        n_old = n
        r = (points - g) @ n
        w = np.exp(-(r ** 2 / k_w ** 2))
        X_k = np.average(points - g - X_prev, axis=0, weights=w)
        vec = (points - g - X_k)
        X_prev = X_k
        C = (vec.T * w) @ vec
        w, v = np.linalg.eig(C)
        v = v.T
        o = w.argsort()
        w = w[o]
        v = v[o]
        n = v[0]
        convg = np.linalg.norm(n_old - n) / np.linalg.norm(n_old)
        if convg < gamma:
            break
    return n

In [ ]:
mu = []
normals = []
for ind, i in tqdm_notebook(list(enumerate(unique))):
    this_cell = np.array(point_cloud_in_numpy[np.where(np.prod(A == i, axis = -1))])
    if (len(this_cell) <= 3):
        P.append(this_cell)
        continue
    mu.append(np.mean(this_cell))
    cov = np.cov(np.array(this_cell).T)
    #print("cov = ", cov)
    w, v = np.linalg.eig(cov)
    v = v.T
    o = w.argsort()
    w = w[o]
    v = v[o]
    #print("values = ", w)
    if (abs(w[0] / w[1]) <= te):
        Q = np.vstack((Q, i))
        Q_points.append(list(this_cell))
        normals.append(IRLS_find_norm(this_cell))
    else:
        P.append(list(this_cell))
Q = Q[1:,]
P = np.concatenate(P)

In [ ]:
Q_points = np.array(Q_points)
normals = np.array(normals)
print(np.shape(normals))

# RANSAC

In [ ]:
def plane(Q, P, Q_points, normals):
    k_max = 50
    nu = 0.95
    delta_d = 0.08
    n = 0
    delta_theta = 0.25
    k = 0
    Psi = np.zeros(len(Q), dtype=np.bool)
    Psi_size = 0
    Psi_points = []
    k = 1
    while k < k_max:
        c = np.random.choice(range(0, len(Q)))
        cell = Q[c]
        points = Q_points[c]
        gk = np.mean(points, axis = 0)
        nk = normals[c]
        Ik = np.zeros(len(Q), dtype=np.bool)
        cnt = 0
        Ik_points = []
        for i, v in enumerate(Q):
            g_i = np.mean(Q_points[i], axis = 0)
            n_i = normals[i]
            d_i = np.dot((g_i - gk), nk) / np.linalg.norm(nk)
            theta_i = 1 - np.abs(np.dot(nk, n_i)) / (np.linalg.norm(nk) * np.linalg.norm(n_i))
            if (np.abs(d_i) < delta_d and theta_i < delta_theta):
    #             print(d_i, theta_i)
    #             print(n_i, nk)
                Ik[i] = 1
                cnt += 1
                Ik_points += Q_points[i]
        if cnt > Psi_size:
            Psi = Ik
            Psi_points = Ik_points
            Psi_size = cnt
            n = nk
            g = gk
            Pn = Psi_size / len(Q)
            k_max = math.ceil(math.log(1 - nu) / math.log(1 - Pn))
        k += 1
#     print(Psi_size)


#     print(Psi_points[0])
#     print(np.shape(Psi_points))
    
    P_mask = np.zeros(len(P), dtype=np.bool)
    for j, i in enumerate(P):
        d_i = np.inner(i - g, n) / np.linalg.norm(n)
        if (np.abs(d_i) < delta_d):
            P_mask[j] = 1 
            Psi_points += [i]
    return n, g, np.array(Psi_points), Q[~Psi], Q_points[~Psi], P[~P_mask], normals[~Psi]
#     print(Psi_points[0])
#     print(np.shape(Psi_points))  

In [ ]:
def sep(arr):
    l = min(z)
    h = max(z)
    mid = (h + l) / 2
    #print(l, mid, h)
    ind = np.searchsorted(z, mid)
    z1 = z[:ind]
    z2 = z[ind:]
    floors = []
    lwth = 10000
    upth = 100000
    #print(len(z1) / (mid - l))
    #print(len(z2) / (h - mid))
    if len(z1) / (mid - l) > upth:
        floors.append((l, mid))
    elif len(z1) / (mid - l) > lwth:
        floors += sep(z1)
    if len(z2) / (h - mid) > upth:
        floors.append((mid, h))
    elif len(z2) / (h - mid) > lwth:
        floors += sep(z2)
    return floors

In [ ]:
Q_old = Q
P_old = P
Q_points_old = Q_points
normals_old = normals
Q = Q_old
P = P_old
Q_points = Q_points_old
normals = normals_old
z = np.sort(point_cloud_in_numpy[:,2])
z = sep(z)
print(z)

In [ ]:
Q = Q_old
P = P_old
Q_points = Q_points_old
normal_points = normals_old
walls_points = np.zeros((0, 3))
walls_colors = np.zeros((0, 3))
walls_normals = np.zeros((0, 3))
floors_points = np.zeros((0, 3))
floors_colors = np.zeros((0, 3))
floors_normals = np.zeros((0, 3))
print(z)
final_f = []
#for i in range(100):
for ind, i in tqdm_notebook(list(enumerate(range(100)))):
    #print(np.shape(normal_points))
    n, g, Psi_points, Q, Q_points, P, normal_points = plane(Q, P, Q_points, normal_points)
    print(n)
#     print(g)
    print(len(Psi_points))
    print(max(Psi_points[:, 2]), min(Psi_points[:, 2]))
    if len(Psi_points) < 10000:
        break
    pcd.points = o3d.utility.Vector3dVector(np.concatenate((floors_points, walls_points)))
    pcd.colors = o3d.utility.Vector3dVector(np.concatenate((floors_colors, walls_colors)))
    theta_i = 1 - np.abs(np.dot([0, 0, 1], n)) / (np.linalg.norm([0, 0, 1]) * np.linalg.norm(n))
    if (theta_i <= 0.15):
        mean_z = np.mean(Psi_points[:,2])
        for j in z:
            if mean_z <= j[1] and mean_z >= j[0]:
                print("1")
                floors_points = np.concatenate((floors_points, Psi_points))
                floors_colors = np.concatenate((floors_colors, np.repeat(np.random.random(3).reshape(1, 3), len(Psi_points), axis = 0)))
                floors_normals = np.concatenate((floors_normals, [n] * len(Psi_points)))
    else:
        print(2)
        walls_points = np.concatenate((walls_points, Psi_points))
        walls_colors = np.concatenate((walls_points, np.repeat(np.random.random(3).reshape(1, 3), len(Psi_points), axis = 0)))
        walls_normals = np.concatenate((walls_normals, [n] * len(Psi_points)))
pcd.points = o3d.utility.Vector3dVector(np.concatenate((floors_points, walls_points)))
pcd.colors = o3d.utility.Vector3dVector(np.concatenate((floors_colors, walls_colors)))
o3d.visualization.draw_geometries([pcd])

In [ ]:
pcd.points = o3d.utility.Vector3dVector(floors_points)
pcd.colors = o3d.utility.Vector3dVector(floors_colors)
o3d.visualization.draw_geometries([pcd])

# Mesh Creation from Walls, Ceiling, and Floor

This will be attempted after the first bit is done. 